In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from keras.utils.np_utils import to_categorical # label数组变为one-hot编码
from keras.models import Sequential
from keras.layers import Flatten, Dense, BatchNormalization, Dropout, Flatten, CuDNNLSTM, CuDNNGRU, Activation
from keras.optimizers import Adam # Adam 优化，加速收敛
from keras.callbacks import ReduceLROnPlateau # tensorboard记录，learning rate decay，模型保存

In [ ]:
from sklearn.model_selection import train_test_split # 用于划分训练集和验证集

In [ ]:
from label_data import get_labeled_data

In [ ]:
seed = 2

In [ ]:
def get_train_val_test(dat_name, lab_name):
    train_x, train_y = get_labeled_data(dat_name, lab_name, 'train_')
    test_x, test_y = get_labeled_data(dat_name, lab_name, 'test_')
    train_y = to_categorical(train_y, num_classes = 3)
    test_y = to_categorical(test_y, num_classes = 3)
    test_x, val_x, test_y, val_y = train_test_split(test_x, 
                                                    test_y, 
                                                    test_size = 0.2, 
                                                    random_state = seed)
    return train_x, train_y, val_x, val_y, test_x, test_y

In [ ]:
train_x, train_y, val_x, val_y, test_x, test_y = get_train_val_test('A1', 'A3')

In [ ]:
UNIT_SIZE = 20
TIME_STEPS = 30
INPUT_SIZE = 20
OUTPUT_SIZE = 3
from keras import backend as K
def model_structure(model):
    model.add(CuDNNGRU(
        units = UNIT_SIZE,
        batch_input_shape=(None, TIME_STEPS, INPUT_SIZE)))
#     model.add(Dense(1))
#     model.add(Activation('relu'))
    model.add(Dense(OUTPUT_SIZE, activation = 'softmax'))
    model.compile(optimizer=Adam(),
                  loss="categorical_crossentropy",
                  metrics=['accuracy'])


In [ ]:
train_sample, train_label, val_x, val_y, test_x, test_y = get_train_val_test('A1', 'A3')

K.set_image_dim_ordering('th')
#生成一个model
model = Sequential()
#model结构见model_structure.py
model_structure(model)

model.summary()

In [ ]:
#batch大小设为128, 训练17轮, 验证集划分设成0.05
model.fit(train_sample, 
          train_label, 
          batch_size=128, 
          epochs=17, 
          verbose=1, 
          validation_data=(val_x, val_y))


In [ ]:
train_label[:10]

In [ ]:
val_y[:10]

In [ ]:
val_x[:2]

In [ ]:
train_sample[:2]